In [1]:
#/********************************************
#* Author: Rahat Ul Ain 
#* Based on chrononet implementation by Kunal Patel
#* location: https://github.com/kunalpatel1793/Neural-Nets-Final-Project
#********************************************/
from pdb import set_trace
import mne
import pandas as pd
import numpy as np
import braindecode 
import torch
from torch import nn
from skorch.callbacks import Checkpoint
from skorch.dataset import ValidSplit,Dataset
from skorch.helper import predefined_split
import math
import os
import h5py
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
from tensorflow.keras.layers import SimpleRNN,LSTM, Dense, Activation, Bidirectional, Flatten, Dropout, Convolution2D, BatchNormalization, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
#tf.config.set_logical_device_configuration(
#        tf.config.list_physical_devices('GPU')[0],
#        [tf.config.LogicalDeviceConfiguration(memory_limit=int(1024*3.9))])

In [4]:
def readDatafromPath(path,windows=1):
	matrix= np.empty((15000, 22), dtype='f')
	for file in os.listdir(path):
		if '.edf' in file:
			f=os.path.join(path, file)
			try:
				edf_file = mne.io.read_raw_edf(f,  eog = ['FP1', 'FP2', 'F3', 'F4',
																	'C3', 'C4',  'P3', 'P4','O1', 'O2','F7', 'F8',
																	'T3', 'T4', 'T5', 'T6','PZ','FZ', 'CZ','A1', 'A2'], verbose='error')
				print(f)
			except:
				continue
			edf_file_down_sampled = edf_file.resample(250, npad = "auto")# set sampling frequency to 250 Hz
			ed = edf_file_down_sampled.to_data_frame(picks = None, index = None, time_format = None, scalings = None,
			copy = True, start = None, stop = None)# converting into dataframe
			Fp1_Fp7 = (ed.loc[: , 'FP1']) - (ed.loc[: , 'F7'])
			FP2_F8 = (ed.loc[: , 'FP2']) - (ed.loc[: , 'F8'])
			F7_T3 = (ed.loc[: , 'F7']) - (ed.loc[: , 'T3'])
			F8_T4 = (ed.loc[: , 'F8']) - (ed.loc[: , 'T4'])
			T3_T5 = (ed.loc[: , 'T3']) - (ed.loc[: , 'T5'])
			T4_T6 = (ed.loc[: , 'T4']) - (ed.loc[: , 'T6'])
			T5_O1 = (ed.loc[: , 'T5']) - (ed.loc[: , 'O1'])
			T6_O2 = (ed.loc[: , 'T6']) - (ed.loc[: , 'O2'])
			A1_T3 = (ed.loc[: , 'A1']) - (ed.loc[: , 'T3'])
			T4_A2 = (ed.loc[: , 'T4']) - (ed.loc[: , 'A2'])
			T3_C3 = (ed.loc[: , 'T3']) - (ed.loc[: , 'C3'])
			C4_T4 = (ed.loc[: , 'C4']) - (ed.loc[: , 'T4'])
			C3_CZ = (ed.loc[: , 'C3']) - (ed.loc[: , 'CZ'])
			CZ_C4 = (ed.loc[: , 'CZ']) - (ed.loc[: , 'C4'])
			FP1_F3 = (ed.loc[: , 'FP1']) - (ed.loc[: , 'F3'])
			FP2_F4 = (ed.loc[: , 'FP2']) - (ed.loc[: , 'F4'])
			F3_C3 = (ed.loc[: , 'F3']) - (ed.loc[: , 'C3'])
			F4_C4 = (ed.loc[: , 'F4']) - (ed.loc[: , 'C4'])
			C3_P3 = (ed.loc[: , 'C3']) - (ed.loc[: , 'P3'])
			C4_P4 = (ed.loc[: , 'C4']) - (ed.loc[: , 'P4'])
			P3_O1 = (ed.loc[: , 'P3']) - (ed.loc[: , 'O1'])
			P4_O2 = (ed.loc[: , 'P4']) - (ed.loc[: , 'O2'])
			data = {
			'Fp1_Fp7': Fp1_Fp7,
			'FP2_F8': FP2_F8,
			'F7_T3': F7_T3,
			'F8_T4': F8_T4,
			'T3_T5': T3_T5,
			'T4_T6': T4_T6,
			'T5_O1': T5_O1,
			'T6_O2': T6_O2,
			'A1_T3': A1_T3,
			'T4_A2': T4_A2,
			'T3_C3': T3_C3,
			'C4_T4': C4_T4,
			'C3_CZ': C3_CZ,
			'CZ_C4': CZ_C4,
			'FP1_F3': FP1_F3,
			'FP2_F4': FP2_F4,
			'F3_C3': F3_C3,
			'F4_C4': F4_C4,
			'C3_P3': C3_P3,
			'C4_P4': C4_P4,
			'P3_O1': P3_O1,
			'P4_O2': P4_O2
			}
			new_data_frame = pd.DataFrame(data, columns = ['Fp1_Fp7', 'FP2_F8', 'F7_T3', 'F8_T4', 'T3_T5', 'T4_T6', 'T5_O1', 'T6_O2', 'A1_T3', 'T4_A2', 'T3_C3', 'C4_T4', 'C3_CZ',
			'CZ_C4', 'FP1_F3', 'FP2_F4', 'F3_C3', 'F4_C4', 'C3_P3', 'C4_P4', 'P3_O1', 'P4_O2'
			])
			fs = edf_file_down_sampled.info['sfreq']
			[row, col] = new_data_frame.shape
			n = math.ceil(row / (15000 - (fs * 5)))
			i = 0
			j = 15000
			#print(f"row:{row},n:{n}")

			for y in range(n - 1):

				#print(f"i:{i},j:{j},y:{y}")
				if y>windows:
					break
				elif y == 0 and j < row:
					example_1 = new_data_frame[0: 15000]
					matrix=np.dstack((matrix,example_1.to_numpy()))
				elif j < row:
					example = new_data_frame[i: j]
					matrix = np.dstack((matrix, example.to_numpy()))
				else :
					try:
						example = new_data_frame[-15000: ]
						matrix = np.dstack((matrix, example.to_numpy()))
					except:
						pass
					finally:
						break
				#Goes to next window with 5 seconds of overlap
				i = int(j - (fs * 5))
				j = int(j + 15000 - (fs * 5))
			#print(f"shape:{matrix.shape}")
	matrix=matrix[:,:,1:]
	return matrix.astype('float32')

In [ ]:
import Chrono_config
print('starting')
normal_train = readDatafromPath(path = Chrono_config.normaldir)
normal_train_dim = normal_train.shape[-1]
# print("normal original dim")
# print(normal_train_dim)
normal_train_zeros = np.zeros(normal_train_dim)
# print("zeros array dim")
# print(normal_train_zeros)
abnormal_train = readDatafromPath(path = Chrono_config.abnormaldir)
abnormal_train_dim = abnormal_train.shape[-1]
#print(abnormal_train_dim)
abnormal_train_ones = np.ones(abnormal_train_dim)
#print(abnormal_train_dim)

train_data = np.dstack((normal_train, abnormal_train))
train_label = np.append(normal_train_zeros, abnormal_train_ones)

train_data = np.swapaxes(train_data,0,2)

bs,t,f = train_data.shape

print(train_data.shape)
print(train_label.shape)
print(train_data.dtype)
print(train_label.dtype)
#enc_labels = to_categorical(train_label, num_classes=2)              
#train_label= enc_labels
print(train_data.shape)
print(train_label.shape)
print(train_data.dtype)
print(train_label.dtype)
print('training labels have been loaded')
del normal_train,normal_train_dim,normal_train_zeros,abnormal_train,abnormal_train_dim,abnormal_train_ones

In [ ]:
# ----------------------CHRONONET Testing-----------------------
from tensorflow.keras.layers import Dense,concatenate,Flatten,GRU,Conv1D
from tensorflow.keras import Model,Input
inputsin= Input(shape=(t,f))
# ------------------First Inception
tower1 = Conv1D(32, 2, strides=2,activation='relu',padding="causal")(inputsin)
tower1 = BatchNormalization()(tower1)
tower2 = Conv1D(32, 4, strides=2,activation='relu',padding="causal")(inputsin)
tower2 = BatchNormalization()(tower2)
tower3 = Conv1D(32, 8, strides=2,activation='relu',padding="causal")(inputsin)
tower3 = BatchNormalization()(tower3)
x = concatenate([tower1,tower2,tower3],axis=2)
x = Dropout(0.45)(x)

# ----------------------Second Inception
tower1 = Conv1D(32, 2, strides=2,activation='relu',padding="causal")(x)
tower1 = BatchNormalization()(tower1)
tower2 = Conv1D(32, 4, strides=2,activation='relu',padding="causal")(x)
tower2 = BatchNormalization()(tower2)
tower3 = Conv1D(32, 8, strides=2,activation='relu',padding="causal")(x)
tower3 = BatchNormalization()(tower3)
x = concatenate([tower1,tower2,tower3],axis=2)
x = Dropout(0.45)(x)

# ----------------------------------Third Inception
tower1 = Conv1D(32, 2, strides=2,activation='relu',padding="causal")(x)
tower1 = BatchNormalization()(tower1)
tower2 = Conv1D(32, 4, strides=2,activation='relu',padding="causal")(x)
tower2 = BatchNormalization()(tower2)
tower3 = Conv1D(32, 8, strides=2,activation='relu',padding="causal")(x)
tower3 = BatchNormalization()(tower3)
x = concatenate([tower1,tower2,tower3],axis=2)
x = Dropout(0.55)(x)

res1 = GRU(32,activation='tanh',return_sequences=True)(x)
res2 = GRU(32,activation='tanh',return_sequences=True)(res1)
res1_2 = concatenate([res1,res2],axis=2)
res3 = GRU(32,activation='tanh',return_sequences=True)(res1_2)
x = concatenate([res1,res2,res3])
x = GRU(32,activation='tanh')(x)

predictions = Dense(2,activation='softmax')(x)
model = Model(inputs=inputsin, outputs=predictions)

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])
print(model.metrics_names)
print(model.summary())

# early stopping
es = EarlyStopping(monitor='val_loss', min_delta=0.01, mode='min', verbose=1, patience=25)                          #patience
mc = ModelCheckpoint('modelbest_acc.hdf5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)        #filepath (save model as)
mces = ModelCheckpoint('modelbest_loss.hdf5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)      #filepath (save model as)
del inputsin,tower1,tower2,tower3,x,res1,res2,res1_2,res3,predictions

In [ ]:
# fit model
hist=model.fit(train_data,train_label,validation_split=0.2,epochs=500,batch_size=1,verbose=1,callbacks=[es, mc,mces],shuffle=True) #epochs #split #
print('The End')

In [5]:
import Chrono_config
print('starting')
normal_eval = readDatafromPath(path = Chrono_config.eval_normaldir,windows=8)
normal_eval_dim = normal_eval.shape[-1]
# print("normal original dim")
# print(normal_eval_dim)
normal_eval_zeros = np.zeros(normal_eval_dim)
# print("zeros array dim")
# print(normal_eval_zeros)

abnormal_eval = readDatafromPath(path = Chrono_config.eval_abnormaldir,windows=8)
abnormal_eval_dim = abnormal_eval.shape[-1]
#print(abnormal_eval_dim)
abnormal_eval_ones = np.ones(abnormal_eval_dim)
#print(abnormal_eval_dim)

eval_data = np.dstack((normal_eval, abnormal_eval))
eval_label = np.append(normal_eval_zeros, abnormal_eval_ones)

eval_data = np.swapaxes(eval_data,0,2)

bs,t,f = eval_data.shape

#enc_labels = to_categorical(eval_label, num_classes=2)
#eval_label= enc_labels
del normal_eval,normal_eval_dim,normal_eval_zeros,abnormal_eval,abnormal_eval_dim,abnormal_eval_ones

starting
E:/NMT_dataset/normal/eval\0000024.edf
E:/NMT_dataset/normal/eval\0000044.edf
E:/NMT_dataset/normal/eval\0000074.edf
E:/NMT_dataset/normal/eval\0000077.edf
E:/NMT_dataset/normal/eval\0000117.edf
E:/NMT_dataset/normal/eval\0000124.edf
E:/NMT_dataset/normal/eval\0000156.edf
E:/NMT_dataset/normal/eval\0000194.edf
E:/NMT_dataset/normal/eval\0000204.edf
E:/NMT_dataset/normal/eval\0000207.edf
E:/NMT_dataset/normal/eval\0000214.edf
E:/NMT_dataset/normal/eval\0000254.edf
E:/NMT_dataset/normal/eval\0000274.edf
E:/NMT_dataset/normal/eval\0000276.edf
E:/NMT_dataset/normal/eval\0000336.edf
E:/NMT_dataset/normal/eval\0000396.edf
E:/NMT_dataset/normal/eval\0000436.edf
E:/NMT_dataset/normal/eval\0000454.edf
E:/NMT_dataset/normal/eval\0000504.edf
E:/NMT_dataset/normal/eval\0000526.edf
E:/NMT_dataset/normal/eval\0000537.edf
E:/NMT_dataset/normal/eval\0000544.edf
E:/NMT_dataset/normal/eval\0000566.edf
E:/NMT_dataset/normal/eval\0000584.edf
E:/NMT_dataset/normal/eval\0000596.edf
E:/NMT_dataset/n

In [ ]:
import scipy
scipy.io.savemat("D:/chrono_eval.mat",{"x":eval_data,"y":eval_label})

In [ ]:
import scipy
mid=len(train_label)//2
scipy.io.savemat("D:/chrono_train_1.mat",{"x":train_data[:mid,:,:],"y":train_label[:mid,:]})
scipy.io.savemat("D:/chrono_train_2.mat",{"x":train_data[mid:,:,:],"y":train_label[mid:,:]})
scipy.io.savemat("D:/chrono_eval.mat",{"x":eval_data,"y":eval_label})

In [2]:
import scipy
import numpy as np
inputs=scipy.io.loadmat("D:/chrono_train_1.mat")
train_data=inputs["x"]
train_label=inputs["y"]
inputs=scipy.io.loadmat("D:/chrono_train_2.mat")
train_data=np.concatenate((train_data,inputs["x"]),axis=0)
train_label=np.concatenate((train_label,inputs["y"]),axis=0)
train_label=train_label.argmax(axis=-1)
bs,t,f = train_data.shape
inputs=scipy.io.loadmat("D:/chrono_eval.mat")
eval_data=inputs["x"]
eval_label=inputs["y"].argmax(axis=-1)
del inputs

In [3]:
class Inception_Block(torch.nn.Module):
  def __init__(self,t=32,out_channels=32,pad=0):
    super().__init__()
    self.tower1= nn.Conv1d(in_channels=t,out_channels=32,kernel_size=2,stride=2,padding=pad)
    self.tower2= nn.Conv1d(in_channels=t,out_channels=32,kernel_size=4,stride=2,padding=1+pad)
    self.tower3= nn.Conv1d(in_channels=t,out_channels=32,kernel_size=8,stride=2,padding=3+pad)
    self.bnorm=nn.BatchNorm1d(32)
    self.relu=nn.ReLU()
    self.dropout = nn.Dropout1d(0.45)

  def forward(self, inputs):
    t1=self.tower1(inputs)
    t1=self.relu(t1)
    t1=self.bnorm(t1)

    t2=self.tower2(inputs)
    t2=self.relu(t2)
    t2=self.bnorm(t2)

    t3=self.tower3(inputs)
    t3=self.relu(t3)
    t3=self.bnorm(t3)

    output=torch.cat([t1,t2,t3],dim=1)
    output=self.dropout(output)
    return output

In [4]:
class ChronoNet(torch.nn.Module):
  def __init__(self,t=32,out_channels=32):
    super().__init__()
    self.block1= Inception_Block(t,out_channels)
    self.block2= Inception_Block(96,out_channels,pad=1)
    self.block3= Inception_Block(96,out_channels)
    self.gru1=nn.GRU(input_size=96,hidden_size=32,batch_first=True)
    self.gru2=nn.GRU(input_size=32,hidden_size=32,batch_first=True)
    self.gru3=nn.GRU(input_size=64,hidden_size=32,batch_first=True)
    self.gru4=nn.GRU(input_size=96,hidden_size=32,batch_first=True)
    self.tanh=nn.Tanh()
    self.classifier = nn.Linear(32,2)
    self.softmax=nn.Softmax(dim=1)

  def forward(self, inputs):

    b1=self.block1(inputs.mT)

    b2=self.block2(b1)

    b3=self.block3(b2)

    res1,_=self.gru1(b3.mT)
    res1=self.tanh(res1)
    res2,_=self.gru2(res1)
    res2=self.tanh(res2)
    res1_2=torch.cat([res1,res2],dim=2)
    res3,_=self.gru3(res1_2)
    res3=self.tanh(res3)
    x=torch.cat([res1,res2,res3],dim=2)
    x,_=self.gru4(x)
    x=self.tanh(x)
    if x.dim()==2:
      x=x[-1,:]
    elif x.dim()==3:
      x=x[:,-1,:]
    output=self.classifier(x)
    output=self.softmax(output)
    return output
#Causal conv1d difficult, causal padding is left padding of dilation_rate*(kernel_size-1) where rate is 1
#Will have to define a class in order to make chrononet
model=ChronoNet(t=f)

In [5]:
a=torch.rand(size=(3,22,15000))
out=model.forward(a)
print(out.shape)

torch.Size([3, 2])


In [6]:
monitor = lambda net: any(net.history[-1, ('valid_accuracy_best','valid_f1_best','valid_loss_best')])
cp=Checkpoint(monitor='valid_f1_best',dirname='model',f_params='ChronoNetbest_param.pkl',f_optimizer='ChronoNetbest_opt.pkl',
              f_history='ChronoNetbest_history.json')
classifier = braindecode.EEGClassifier(
        model,
        criterion=nn.CrossEntropyLoss(),
        optimizer=torch.optim.AdamW,
        train_split=predefined_split(Dataset(eval_data,eval_label)),
        optimizer__lr=0.00003,
        iterator_train__shuffle=True,
        batch_size=16,
        device="cuda",
        callbacks=["accuracy","f1",'roc_auc',cp],
        warm_start=True,
        )
classifier.initialize()

<class 'braindecode.classifier.EEGClassifier'>[initialized](
  module_=ChronoNet(
    (block1): Inception_Block(
      (tower1): Conv1d(15000, 32, kernel_size=(2,), stride=(2,))
      (tower2): Conv1d(15000, 32, kernel_size=(4,), stride=(2,), padding=(1,))
      (tower3): Conv1d(15000, 32, kernel_size=(8,), stride=(2,), padding=(3,))
      (bnorm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (dropout): Dropout1d(p=0.45, inplace=False)
    )
    (block2): Inception_Block(
      (tower1): Conv1d(96, 32, kernel_size=(2,), stride=(2,), padding=(1,))
      (tower2): Conv1d(96, 32, kernel_size=(4,), stride=(2,), padding=(2,))
      (tower3): Conv1d(96, 32, kernel_size=(8,), stride=(2,), padding=(4,))
      (bnorm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (dropout): Dropout1d(p=0.45, inplace=False)
    )
    (block3): Inception_Block(
      (tower1): Conv1d(96, 32,

In [7]:
classifier.load_params(
        f_params=f'model/ChronoNetbest_param.pkl', f_optimizer=f'model/ChronoNetbest_opt.pkl', f_history=f'model/ChronoNetbest_history.json')
print("Paramters Loaded")

Paramters Loaded


In [10]:
classifier.fit(train_data,train_label,epochs=3)

     10            0.9519      0.8229        0.4194           0.8942            0.5565      0.2293        0.7476           0.6483        56.9080
     11            0.9562      0.8402        0.4054           0.9042            0.5521      0.2104        0.7544           0.6396        56.9490
     12            0.9445      0.7868        0.3973           0.9126            0.5295      0.1008        0.7791           0.6273        56.7230


<class 'braindecode.classifier.EEGClassifier'>[initialized](
  module_=ChronoNet(
    (block1): Inception_Block(
      (tower1): Conv1d(15000, 32, kernel_size=(2,), stride=(2,))
      (tower2): Conv1d(15000, 32, kernel_size=(4,), stride=(2,), padding=(1,))
      (tower3): Conv1d(15000, 32, kernel_size=(8,), stride=(2,), padding=(3,))
      (bnorm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (dropout): Dropout1d(p=0.45, inplace=False)
    )
    (block2): Inception_Block(
      (tower1): Conv1d(96, 32, kernel_size=(2,), stride=(2,), padding=(1,))
      (tower2): Conv1d(96, 32, kernel_size=(4,), stride=(2,), padding=(2,))
      (tower3): Conv1d(96, 32, kernel_size=(8,), stride=(2,), padding=(4,))
      (bnorm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (dropout): Dropout1d(p=0.45, inplace=False)
    )
    (block3): Inception_Block(
      (tower1): Conv1d(96, 32,

In [9]:
predicted_labels=classifier.predict(eval_data)
accuracy=np.mean(predicted_labels==eval_label)
print(f"Accuracy:{accuracy}")
tp=np.sum(predicted_labels*eval_label)
precision=tp/np.sum(predicted_labels)
recall=tp/np.sum(eval_label)
f1=2*precision*recall/(precision+recall)
print(f"F1-Score:{f1}")

Accuracy:0.5367847411444142
F1-Score:0.17475728155339806


In [ ]:
print('testing')
# load the saved accuracy model
savedmodel = load_model('model3flipped_loss.hdf5')                      # model name
print('model loaded')
test_score = savedmodel.evaluate(test_data, test_labels, batch_size=32)
print ("Evaluation loss and Evaluation accuracy for best accuracy model is: ", test_score)
print ('The End')